<a href="https://colab.research.google.com/github/Monmon891Awesome/swot/blob/master/Final_Training_Yolo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec  4 11:30:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 18.5 MB/s eta 0:00:00


In [5]:
# Install PyTorch with GPU (CUDA) support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device Count: {torch.cuda.device_count()}")

CUDA Available: True
CUDA Device Count: 1


In [7]:
import zipfile

zip_path = '/content/drive/My Drive/THESIS FILES/UDsets/Unified.zip'
extract_path = '/content/dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
import os
print(f"Extracted files: {os.listdir(extract_path)}")

Extracted files: ['__MACOSX', 'Unified DS']


In [8]:
yaml_content = """
train: /content/dataset/Unified DS/train/images
val: /content/dataset/Unified DS/valid/images
test: /content/dataset/Unified DS/test/images

nc: 4
names: [Pothole, Pole, Broken_Pole, Crack]
"""
with open('/content/dataset/Unified DS/forcolabdata.yaml', 'w') as yaml_file:
    yaml_file.write(yaml_content)

In [9]:
import os

unified_ds_path = '/content/dataset/Unified DS'
print(f"Contents of 'Unified DS': {os.listdir(unified_ds_path)}")

Contents of 'Unified DS': ['unifieddata.yaml', 'valid', 'train', '.DS_Store', 'test', 'forcolabdata.yaml']


In [10]:
with open('/content/dataset/Unified DS/forcolabdata.yaml', 'r') as yaml_file:
    print(yaml_file.read())


train: /content/dataset/Unified DS/train/images
val: /content/dataset/Unified DS/valid/images
test: /content/dataset/Unified DS/test/images

nc: 4
names: [Pothole, Pole, Broken_Pole, Crack]



In [11]:
from ultralytics import YOLO

model1 = YOLO('yolo11n.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 67.7MB/s]


In [13]:
model1.train(
    data='/content/dataset/Unified DS/forcolabdata.yaml',  # Path to data.yaml
    epochs=100,                 # Training for 150 epochs
    imgsz=1088,                 # Image size
    batch=64,                  # Batch size for A100
    device='0',                 # Use GPU (A100)
    cache=True,               # Cache dataset on disk
    amp=True,                   # Automatic mixed precision
    patience=30,                # Early stopping after 10 epochs without improvement
    save_period=10,             # Save weights every 10 epochs
    workers=4                   # Number of dataloader workers
)

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/dataset/Unified DS/forcolabdata.yaml, epochs=100, time=None, patience=30, batch=64, imgsz=1088, save=True, save_period=10, cache=True, device=0, workers=4, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

train: Scanning /content/dataset/Unified DS/train/labels.cache... 4927 images, 882 backgrounds, 0 corrupt: 100%|██████████| 4927/4927 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 7652, len(boxes) = 8903. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (16.3GB RAM): 100%|██████████| 4927/4927 [00:05<00:00, 883.08it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all        909       1489      0.688      0.647      0.657      0.402



100 epochs completed in 1.466 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.17it/s]


                   all        909       1489      0.688      0.645      0.656      0.401
               Pothole        153        477      0.721      0.633      0.655      0.478
                  Pole         13         22      0.715      0.909      0.828      0.632
           Broken_Pole        400        629      0.617       0.25      0.356      0.126
                 Crack        296        361        0.7      0.788      0.786      0.367
Speed: 0.4ms preprocess, 1.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x794eed0380a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0